In [1]:
from selenium import webdriver
import time
import pandas as pd 

In [2]:
def onlynumbers(gimmestring):
    num = ["0","1","2","3","4","5","6","7","8","9"]
    val = ""
    for letter in gimmestring:
        if(letter in num):
            val = val+letter
    return val

In [3]:
# Esta função cria um dataframe com as informações dentro da página de busca do datajus

# datajus/diarios/

# title = titulo do artigo
# publisher = publicante do artigo
# body = conteúdo da página encontrada com a keyword referenciada
# link = link do artigo 

def page_dataframe():
    df = pd.DataFrame()

    titleL = []
    publisherL = []
    bodyL = []
    linkL = []
    
    #document_folder é a DIV dentro do HTML que representa todos os documentos dentro  da página
    #document_children serão os elementos filhos de 1º grau do elemento document_folder (cada um dos documentos)
    
    #usaremos isso para criar a condicional de visualização do crawler
    document_folder = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]")
    document_children = document_folder.find_elements_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/*")
    
    for i in document_children:
        print()
    
    x = 1
    for i in document_children:
        #se o atributo "class" do elemento filho se chamar 'searchpaginator', signfica que ele é o page_selector
        if((i.get_attribute('class')) == 'Pagination pagination SearchPaginator'):
            break
        publisher = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/div[1]".format(x))
        if 'Tribunal' not in publisher.text and 'Justiça' not in publisher.text:
            title = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/h2".format(x))
            body = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/div[2]".format(x))
            link = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/h2/a".format(x))
            link_text = link.get_attribute("href")
            
            titleL.append(title.text)
            publisherL.append(publisher.text)
            bodyL.append(body.text)
            linkL.append(link_text)
        x = x+1
    
    df["title"] = titleL
    df["url"] = linkL
    df["publisher"] = publisherL
    df["body"] = bodyL
    
    return df, linkL

In [4]:
def searchcount():
    count = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[1]/div[1]/div[1]")
    
    return count.text

In [5]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

path = "chromedriver.exe"

driver = webdriver.Chrome(executable_path=path, options=options)
time.sleep(3)

In [6]:
# esta celula ainda está sujeita a alteração, já que a KEYWORD pode ser alterado

# dentro de KEYWORD, se coloca a palavra a ser pesquisada
# para usar mais de um termo em KEYWORD, usar mesmas regras do google:
# "covid +suspensão" (encontra artigos que contenham o texto "covid" e também o texto "suspensão")

# filt é a ordem em que os resultados serão mostrados (não é tão relevante já que pegaremos info em massa)
url = 'https://www.jusbrasil.com.br/diarios/'

keyword = "covid+fechamento+decreta"
filt = "data&l=1dia"

In [7]:
# loop para entrar em cada uma das páginas do datajus

# Este loop ainda não está completo, por isso deixamos o range baixo (apenas para retirar o dataframe da primeira pagina)

df_all = pd.DataFrame()

#range maximo de 50 paginas
for i in range(1,50):
    page = i
    search = "busca?q="+keyword+"&p="+str(page)+"&o="+filt
    if(page==1):
        search = "busca?q="+keyword+"&o="+filt
    
    driver.get(url+search)
    
    if(page==1):
        #searchcount é o numero de resultados retornados pelo datajus
        result = searchcount()
        
    if(len(df_all)==result):
        df_all.reset_index(drop=True)
        break
    
    time.sleep(2)
    
    df, linkL = page_dataframe()
    
    df_all = df_all.append(df)
    #loop dentro de cada um dos links que estão dentro da página, para entrar no artigo e extrair os dados
    count=0
    for link in linkL:
        time.sleep(2)
        driver.get(link)
        body = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div/div[2]/div/div[1]")
        
        ser = pd.Series()
        ser["body"] = body.text
        
        df.loc[count]["body"] = ser["body"]
        count=count+1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='app-root']/div/div/div[1]/div[2]/div[1]/div[1]/div[1]"}
  (Session info: headless chrome=80.0.3987.149)


In [ ]:
df_all.to_csv(r'vraul.csv')

In [ ]:
df_all